In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [6]:
# creating supervised learning imports

import pandas as pd

from sklearn.model_selection import train_test_split

<IPython.core.display.Javascript object>

In [3]:
final_df = pd.read_csv("./cleaned_data.csv")
final_df.head()

,event_id,is_active,timestamp,longitude,latitude,category,device_model,gender,age,group,english_phone_brand
0,6,1,2016-05-01 00:27:21,0.0,0.0,Property Industry 1.0,Mate 7,M,19,M22-,huawei
1,6,1,2016-05-01 00:27:21,0.0,0.0,Industry tag,Mate 7,M,19,M22-,huawei
2,6,1,2016-05-01 00:27:21,0.0,0.0,Relatives 1,Mate 7,M,19,M22-,huawei
3,6,1,2016-05-01 00:27:21,0.0,0.0,Property Industry 2.0,Mate 7,M,19,M22-,huawei
4,6,1,2016-05-01 00:27:21,0.0,0.0,IM,Mate 7,M,19,M22-,huawei


<IPython.core.display.Javascript object>

In [14]:
# Creating dummy variables for categorical features
pd.get_dummies(
    final_df,
    columns=["category", "device_model", "gender", "group", "english_phone_brand"],
    drop_first=True,
)

MemoryError: Unable to allocate 24.3 GiB for an array with shape (52262203, 500) and data type uint8

<IPython.core.display.Javascript object>

In [8]:
# splitting up the data and choosing gender as first target variable
X = final_df.drop(columns=["gender"])
y = final_df["gender"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=13, stratify=y
)

<IPython.core.display.Javascript object>